## Use Case 3

Train a random forest to predict "Health" from "Hours worked per week", "Age", "Marital Status", "Occupation", "Industry", "Region".

(This notebook uses the same structure as the other use case notebooks).

In [1]:
import itertools
import json
import numpy as np
import os
import pandas as pd
import pickle
import tqdm
import tqdm.notebook

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [3]:
# Loads of warnings coming from the sub-methods, which we don't care about.
import warnings
warnings.filterwarnings("ignore")

In [4]:
from utils import load_data, load_synthetic_datasets, metadata, custom_ipf, custom_mst

2022-11-03 16:17:11.822978: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


Load the data.

In [5]:
teaching_file_train = load_data(train=True, test=False)
teaching_file_test = load_data(train=False, test=True)

In [6]:
output_size = len(teaching_file_train) + len(teaching_file_test)

### Use case 

What columns are used in the task? What marginals should be preserved for this task?

In [7]:
columns = ["Marital Status", "Occupation", "Industry", "Region", "Hours worked per week", "Age", "Health"]

In [8]:
# Example: use all 2-way marginals.
num_columns = len(columns)

acceptable_marginals = list(itertools.combinations(range(num_columns), 2))

In [9]:
[[columns[i] for i in c] for c in acceptable_marginals]

[['Marital Status', 'Occupation'],
 ['Marital Status', 'Industry'],
 ['Marital Status', 'Region'],
 ['Marital Status', 'Hours worked per week'],
 ['Marital Status', 'Age'],
 ['Marital Status', 'Health'],
 ['Occupation', 'Industry'],
 ['Occupation', 'Region'],
 ['Occupation', 'Hours worked per week'],
 ['Occupation', 'Age'],
 ['Occupation', 'Health'],
 ['Industry', 'Region'],
 ['Industry', 'Hours worked per week'],
 ['Industry', 'Age'],
 ['Industry', 'Health'],
 ['Region', 'Hours worked per week'],
 ['Region', 'Age'],
 ['Region', 'Health'],
 ['Hours worked per week', 'Age'],
 ['Hours worked per week', 'Health'],
 ['Age', 'Health']]

Define the task of interest (analysis on data).

In [10]:
numeric = ["Hours worked per week", "Age"]
categoric = ["Marital Status", "Occupation", "Industry", "Region"]
target = "Health"

In [11]:
# Process the test file in the same format.
df = teaching_file_test.copy()
df = df[[*numeric, *categoric, target]].dropna()

X_test_df, y_test_df = df.drop(target, axis=1), df[target]
X_test_df[numeric] = X_test_df[numeric].astype(int)
X_test_df[categoric] = X_test_df[categoric].astype(str)

In [12]:
def task(dataset, split_to_estimate_accuracy = True):
    # Get the categories from the metadata.
    categories = {m["name"]: m["representation"] for m in metadata if m["name"] in categoric}
    
    # Define the encoder and model (random forest with default parameters).
    encoder = OneHotEncoder(categories=[categories[m] for m in categoric])
    preprocessor = ColumnTransformer(
        [
            ("encode", encoder, categoric),
            ("rescale", StandardScaler(), numeric)
        ],
        remainder="passthrough",
    )
    forest = RandomForestClassifier(
        random_state=0, n_jobs=-1, class_weight="balanced_subsample"
    )
    pipe = Pipeline([("preprocess", preprocessor), ("model", forest)])
    
    # Load the dataset, divide into train/test (to evaluate).
    df = dataset.copy()
    df = df[[*numeric, *categoric, target]].dropna()
    X, y = df.drop(target, axis=1), df[target]
    X[numeric] = X[numeric].astype(int)
    X[categoric] = X[categoric].astype(str)
    
    # Only divide the dataset in training and testing *if* required. This is because synthetic
    # datasets are of larger size, and can be split precisely for training+testing.
    if split_to_estimate_accuracy:
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)
    else:
        X_train = X
        y_train = y
        # Use the real test dataset (only for real data!).
        X_test = X_test_df
        y_test = y_test_df
    
    # Apply the pipeline, and do predictions on the test set.
    print("Training model...")
    model = pipe.fit(X_train, y_train)
    print("Evaluating model...")
    y_pred = model.predict(X_test)
    
    # Get predictions on the real test set.
    y_real_est = model.predict(X_test_df).astype(str)

    # Evaluate feature importance on test.
    print("Evaluating feature importance...")
    importances = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=0, n_jobs=-1)
    
    # Return the model + the y_pred/y_true to evaluate + estimated importances.
    return (y_pred, y_test, y_real_est, y_test_df, importances)

In [13]:
## SOURCE: https://rosettacode.org/wiki/Jaro_similarity#Python

def jaro(s, t):
    '''Jaro distance between two strings.'''
    s_len = len(s)
    t_len = len(t)

    if s_len == 0 and t_len == 0:
        return 1

    match_distance = (max(s_len, t_len) // 2) - 1

    s_matches = [False] * s_len
    t_matches = [False] * t_len

    matches = 0
    transpositions = 0

    for i in range(s_len):
        start = max(0, i - match_distance)
        end = min(i + match_distance + 1, t_len)

        for j in range(start, end):
            if t_matches[j]:
                continue
            if s[i] != t[j]:
                continue
            s_matches[i] = True
            t_matches[j] = True
            matches += 1
            break

    if matches == 0:
        return 0

    k = 0
    for i in range(s_len):
        if not s_matches[i]:
            continue
        while not t_matches[k]:
            k += 1
        if s[i] != t[k]:
            transpositions += 1
        k += 1

    return ((matches / s_len) +
            (matches / t_len) +
            ((matches - transpositions / 2) / matches)) / 3

### Generating tailored datasets

In [14]:
master_seed = 42387342

In [15]:
num_runs = 5

In [16]:
np.random.seed(master_seed)
seeds = np.random.randint(np.iinfo(np.int16).max, size=num_runs)

IPF

In [17]:
for seed in seeds:
    custom_ipf(teaching_file_train, columns, "use_case_3_ipf", seed, acceptable_marginals, output_size)

MST

In [18]:
for seed in seeds:
    custom_mst(teaching_file_train, columns, "use_case_3_mst", seed, acceptable_marginals, output_size)

### Loading synthetic datasets

Load generated datasets that are agnostic to the task, as well as the datasets from the previous step.

In [19]:
methods = [
    "use_case_3_ipf",
    "use_case_3_mst",
    "MST_eps1000",
    "CTGAN_10epochs",
    "PATEGAN_eps1000",
    "PrivBayes_eps1000",
    "SYNTHPOP"
]

### Summarising the results

Little bit of memoising: this loads results previously computed from the disk. If no such results are found, both are re-computed.

In [20]:
try:
    with open("use_case_3_new.pickle", "rb") as ff:
        task_real_data, results = pickle.load(ff)
except Exception as err:
    print(err)
    # If relevant, run the task on the real data.
    task_real_data = task(teaching_file_train, split_to_estimate_accuracy=False)
    results = {}

In [21]:
synthetic_datasets = load_synthetic_datasets(methods)

Perform the task on each dataset. This method is re-entrant: it will not recompute previously obtained results.

In [22]:
for method in tqdm.notebook.tqdm(methods):
    # Prevent re-computation of the task.
    if method in results and len(results[method]) == len(synthetic_datasets[method]):
        continue
    print(method)
    results[method] = L = results.get(method, [])
    for ds in synthetic_datasets[method][len(L):]:
        L.append(task(ds))

  0%|          | 0/7 [00:00<?, ?it/s]

use_case_3_ipf
Training model...
Evaluating model...
Evaluating feature importance...
Training model...
Evaluating model...
Evaluating feature importance...
Training model...
Evaluating model...
Evaluating feature importance...
use_case_3_mst
Training model...
Evaluating model...
Evaluating feature importance...
Training model...
Evaluating model...
Evaluating feature importance...
Training model...
Evaluating model...
Evaluating feature importance...
MST_eps1000
Training model...
Evaluating model...
Evaluating feature importance...
Training model...
Evaluating model...
Evaluating feature importance...
Training model...
Evaluating model...
Evaluating feature importance...
Training model...
Evaluating model...
Evaluating feature importance...
CTGAN_10epochs
Training model...
Evaluating model...
Evaluating feature importance...
Training model...
Evaluating model...
Evaluating feature importance...
Training model...
Evaluating model...
Evaluating feature importance...
Training model...
Ev

Save these task results to disk to remove the need for repeated expensive computations.

In [26]:
with open("use_case_3_new.pickle", "wb") as ff:
    pickle.dump((task_real_data, results), ff)

#### Accuracy report

In [27]:
print("Accuracy trained on _real_ data:", accuracy_score(task_real_data[1], task_real_data[2]))
print()

for method in methods:
    print('===', method, '===')
    acc_method_real = 0
    acc_method_est = 0
    for task_result in results[method]:
        y_pred, y_test, y_real_est, y_test_df, _ = task_result
        ar = accuracy_score(y_test_df, y_real_est)
        ae = accuracy_score(y_test, y_pred)
        acc_method_real += ar
        acc_method_est += ae
    print('\t Accuracy of classifier:  ', acc_method_real / len(results[method]))
    print('\t Accuracy estimated on SD:', acc_method_est / len(results[method]))
    print()

Accuracy trained on _real_ data: 0.44075471698113206

=== use_case_3_ipf ===
	 Accuracy of classifier:   0.4344853005704256
	 Accuracy estimated on SD: 0.43575921817518043

=== use_case_3_mst ===
	 Accuracy of classifier:   0.4381254936375603
	 Accuracy estimated on SD: 0.36953438737397853

=== MST_eps1000 ===
	 Accuracy of classifier:   0.2583905221588416
	 Accuracy estimated on SD: 0.265907495296133

=== CTGAN_10epochs ===
	 Accuracy of classifier:   0.32933742869679683
	 Accuracy estimated on SD: 0.382936897975231

=== PATEGAN_eps1000 ===
	 Accuracy of classifier:   0.21041509433962266
	 Accuracy estimated on SD: 0.45985003791176393

=== PrivBayes_eps1000 ===
	 Accuracy of classifier:   0.21450109697235628
	 Accuracy estimated on SD: 0.22177118144289368

=== SYNTHPOP ===
	 Accuracy of classifier:   0.4368442299254059
	 Accuracy estimated on SD: 0.4726277064786992



#### Feature importance report

In [30]:
print("Feature importance for real model:", task_real_data[-1]["importances_mean"])
optimal_ordering = np.argsort(task_real_data[-1]["importances_mean"])
print("Feature ordering based on importance:", optimal_ordering)
print()

for method in methods:
    print('===', method, '===')
    importance = 0
    avg_distance = 0
    # TODO: average 
    for task_result in results[method]:
        this_imp = task_result[-1]["importances_mean"]
        this_order = np.argsort(this_imp)
        importance += this_imp
        avg_distance += jaro(''.join(optimal_ordering.astype(str)), ''.join(this_order.astype(str)))

    print("\tFeature importance:    ", importance/len(results[method]))
    ordering = np.argsort(importance)
    print("\tOrdering of importance:", ordering)
    similarity = jaro(''.join(optimal_ordering.astype(str)), ''.join(ordering.astype(str)))
#     print("\tSimilarity of ordering:", similarity)
    print("\tAverage similarity oo: ", avg_distance / len(results[method]))
    print()

Feature importance for real model: [ 0.0377534   0.14630452  0.02841597 -0.00124616 -0.00530584  0.00362089]
Feature ordering based on importance: [4 3 5 2 0 1]

=== use_case_3_ipf ===
	Feature importance:     [ 0.0351223   0.14259864  0.00758741 -0.00930762 -0.01488275  0.00031102]
	Ordering of importance: [4 3 5 2 0 1]
	Average similarity oo:  0.9777777777777779

=== use_case_3_mst ===
	Feature importance:     [-0.00529431  0.08425904 -0.03289154 -0.06495759 -0.07246988 -0.05781361]
	Ordering of importance: [4 3 5 2 0 1]
	Average similarity oo:  1.0

=== MST_eps1000 ===
	Feature importance:     [ 0.00984695 -0.00254599 -0.03797228 -0.10608245 -0.08438035  0.00111362]
	Ordering of importance: [3 4 2 1 5 0]
	Average similarity oo:  0.8333333333333334

=== CTGAN_10epochs ===
	Feature importance:     [-0.00766927  0.02026945  0.01661897 -0.02540832 -0.03750807  0.01027296]
	Ordering of importance: [4 3 0 5 2 1]
	Average similarity oo:  0.8922222222222222

=== PATEGAN_eps1000 ===
	Feature

Baseline for Jaro distance.

In [44]:
distances = []
for _ in range(100000):
    x = "123456"
    y = list(x)
    np.random.shuffle(y)
    y2 = list(x)
    np.random.shuffle(y2)
    distances.append(jaro(''.join(y2), ''.join(y)))

In [45]:
np.mean(distances)

0.6958849999999999